In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/imdb-dataset-of-50k-movie-reviews/IMDB Dataset.csv


In [2]:
df = pd.read_csv('/kaggle/input/imdb-dataset-of-50k-movie-reviews/IMDB Dataset.csv')

In [3]:
df.head()

,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive


<h3>Lowercase</h3> <br> In order to perform basic cleanup one of the major steps could be to convert data into lowercase for example "basically" and "Basically" could be considered two different words by the machine in further stages like tokenization, increasing the amount of work 

In [4]:
#converts the row at index 4
df['review'][4].lower()

'petter mattei\'s "love in the time of money" is a visually stunning film to watch. mr. mattei offers us a vivid portrait about human relations. this is a movie that seems to be telling us what money, power and success do to people in the different situations we encounter. <br /><br />this being a variation on the arthur schnitzler\'s play about the same theme, the director transfers the action to the present time new york where all these different characters meet and connect. each one is connected in one way, or another to the next person, but no one seems to know the previous point of contact. stylishly, the film has a sophisticated luxurious look. we are taken to see how these people live and the world they live in their own habitat.<br /><br />the only thing one gets out of all these souls in the picture is the different stages of loneliness each one inhabits. a big city is not exactly the best place in which human relations find sincere fulfillment, as one discerns is the case wit

In [5]:
#to convert the whole dataset and assign it to df 
df['review'] = df['review'].str.lower()
df

,review,sentiment
0,one of the other reviewers has mentioned that ...,positive
1,a wonderful little production. <br /><br />the...,positive
2,i thought this was a wonderful way to spend ti...,positive
3,basically there's a family where a little boy ...,negative
4,"petter mattei's ""love in the time of money"" is...",positive
...,...,...
49995,i thought this movie did a down right good job...,positive
49996,"bad plot, bad dialogue, bad acting, idiotic di...",negative
49997,i am a catholic taught in parochial elementary...,negative
49998,i'm going to have to disagree with the previou...,negative


# Remove HTML Tags
HTML tags are used to make the browser understand but they are of no need here in analysis

In [6]:
import re
def remove_html_tags(text):
    pattern = re.compile('<.*?>')
    return pattern.sub(r'', text)

In [7]:
text = '<html><body><p>Hello this is to test the function</p></body></html>'

In [8]:
remove_html_tags(text)

'Hello this is to test the function'

In [9]:
#applying our function to dataset
df['review'] = df['review'].apply(remove_html_tags)
df['review']

0        one of the other reviewers has mentioned that ...
1        a wonderful little production. the filming tec...
2        i thought this was a wonderful way to spend ti...
3        basically there's a family where a little boy ...
4        petter mattei's "love in the time of money" is...
                               ...                        
49995    i thought this movie did a down right good job...
49996    bad plot, bad dialogue, bad acting, idiotic di...
49997    i am a catholic taught in parochial elementary...
49998    i'm going to have to disagree with the previou...
49999    no one expects the star trek movies to be high...
Name: review, Length: 50000, dtype: object

In [10]:
df['review'][4]

'petter mattei\'s "love in the time of money" is a visually stunning film to watch. mr. mattei offers us a vivid portrait about human relations. this is a movie that seems to be telling us what money, power and success do to people in the different situations we encounter. this being a variation on the arthur schnitzler\'s play about the same theme, the director transfers the action to the present time new york where all these different characters meet and connect. each one is connected in one way, or another to the next person, but no one seems to know the previous point of contact. stylishly, the film has a sophisticated luxurious look. we are taken to see how these people live and the world they live in their own habitat.the only thing one gets out of all these souls in the picture is the different stages of loneliness each one inhabits. a big city is not exactly the best place in which human relations find sincere fulfillment, as one discerns is the case with most of the people we 

# Remove URLS
Remove URLS using regular expression pattern : https?://\S+|www\.\S+

In [11]:
 def remove_urls(text):
        pattern = re.compile('https?://\S+|www\.\S+')
        return pattern.sub(r'', text)
        

In [12]:
text1 = 'check out my notebook https://www.kaggle.com/meghashivhare/text-preprocessing'
text2 = 'check out my notebook http://www.kaggle.com/meghashivhare/text-preprocessing'
text3 = 'search on google - www.google.com'

In [13]:
remove_urls(text1)

'check out my notebook '

In [14]:
remove_urls(text2)

'check out my notebook '

In [15]:
remove_urls(text3)

'search on google - '

# Remove Punctuations
Now we will remove punctuations (! , # etc) <br>
Reason <br>
Tokenization stage - <br>
Case 1 - Machine will treat words, punctuations separately where things like punctuations might be of no use (this might differ according to the project you're making) and will make the document lengthy. <br>
example - Hello! how are you? <br>
will be treated as <br> 1-hello <br> 2-! <br> 3-how <br>.....and so on <br> where storing ! and ? makes no sense <br>
Case 2 - Machine will treat punctuations with word as one <br>
above example will be treated as <br>1-hello!<br> 2-how<br> 3-are <br>4-you? <br>
but now hello and hello? will be considered as 2 different tokens further confusing the model

In [16]:
import string,time
string.punctuation

'!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~'

In [17]:
 exclude = string.punctuation

In [18]:
#function made from scratch
def remove_punc(text):
    for char in exclude:
        text = text.replace(char,'')
    return text

In [19]:
text = "Some? string, with! punctuations#"

In [20]:
remove_punc(text)

'Some string with punctuations'

In [21]:
#let's calculate the time taken 
start = time.time()
print(remove_punc(text))
time1 = time.time() - start
print(time1*5000)

Some string with punctuations
1.5115737915039062


In [22]:
#the time taken is more hence let's try another technique
def remove_punc1(text):
    return text.translate(str.maketrans('','',exclude))

In [23]:
start = time.time()
print(remove_punc1(text))
time1 = time.time() - start
print(time1*5000)

Some string with punctuations
0.8273124694824219


In [24]:
df['review'] = df['review'].apply(remove_punc1)
df['review'][4]

'petter matteis love in the time of money is a visually stunning film to watch mr mattei offers us a vivid portrait about human relations this is a movie that seems to be telling us what money power and success do to people in the different situations we encounter this being a variation on the arthur schnitzlers play about the same theme the director transfers the action to the present time new york where all these different characters meet and connect each one is connected in one way or another to the next person but no one seems to know the previous point of contact stylishly the film has a sophisticated luxurious look we are taken to see how these people live and the world they live in their own habitatthe only thing one gets out of all these souls in the picture is the different stages of loneliness each one inhabits a big city is not exactly the best place in which human relations find sincere fulfillment as one discerns is the case with most of the people we encounterthe acting i

# Chat word Treatment
Converting social media abbreviations like lmao, rofl, asap back to orginal forms using python dictonary 

In [25]:
chat_words = { "AFAIK":"As Far As I Know", "AFK" : "Away From Keyboard",
"ASAP":"As Soon As Possible",
"ATK":"At The Keyboard",
"BRB":"Be Right Back",
"B4N":"Bye For Now",
"TTYL":"Talk To You Later" } 

In [26]:
chat_words

{'AFAIK': 'As Far As I Know',
 'AFK': 'Away From Keyboard',
 'ASAP': 'As Soon As Possible',
 'ATK': 'At The Keyboard',
 'BRB': 'Be Right Back',
 'B4N': 'Bye For Now',
 'TTYL': 'Talk To You Later'}

In [27]:
def chat_conversion(text):
    new_text = []
    for w in text.split():
        if w.upper() in chat_words:
            new_text.append(chat_words[w.upper()])
        else:
            new_text.append(w)
#     return " "..join(new_text)
    return " ".join(new_text)

In [28]:
chat_conversion("I'll call you asap")

"I'll call you As Soon As Possible"

# Spelling Correction

In [29]:
from textblob import TextBlob
incorrect_text = 'soo basicaally this is a sttement with inncorrect words'
txtBlob = TextBlob(incorrect_text)
txtBlob.correct().string

'so basically this is a statement with incorrect words'

# Removing Stop words
Words that are used in sentence formation (like A,the,and etc) but have no meaning and might not be useful for tasks like sentiment analysis altough for tasks like parts of speech tagging stop words might be useful

In [30]:
from nltk.corpus import stopwords

In [31]:
stopwords.words('english')

['i',
 'me',
 'my',
 'myself',
 'we',
 'our',
 'ours',
 'ourselves',
 'you',
 "you're",
 "you've",
 "you'll",
 "you'd",
 'your',
 'yours',
 'yourself',
 'yourselves',
 'he',
 'him',
 'his',
 'himself',
 'she',
 "she's",
 'her',
 'hers',
 'herself',
 'it',
 "it's",
 'its',
 'itself',
 'they',
 'them',
 'their',
 'theirs',
 'themselves',
 'what',
 'which',
 'who',
 'whom',
 'this',
 'that',
 "that'll",
 'these',
 'those',
 'am',
 'is',
 'are',
 'was',
 'were',
 'be',
 'been',
 'being',
 'have',
 'has',
 'had',
 'having',
 'do',
 'does',
 'did',
 'doing',
 'a',
 'an',
 'the',
 'and',
 'but',
 'if',
 'or',
 'because',
 'as',
 'until',
 'while',
 'of',
 'at',
 'by',
 'for',
 'with',
 'about',
 'against',
 'between',
 'into',
 'through',
 'during',
 'before',
 'after',
 'above',
 'below',
 'to',
 'from',
 'up',
 'down',
 'in',
 'out',
 'on',
 'off',
 'over',
 'under',
 'again',
 'further',
 'then',
 'once',
 'here',
 'there',
 'when',
 'where',
 'why',
 'how',
 'all',
 'any',
 'both',
 'each

In [32]:
def remove_stopwords(text):
    new_text =[]
    for word in text.split():
        if word in stopwords.words('english'):
            new_text.append('')
        else:
            new_text.append(word)
    x = new_text[:]
    new_text.clear()
    return " ".join(x)

In [33]:
remove_stopwords('probably hence allowing you to modify your copy without damaging the original, the reason this also works for strings is that in Python, Strings are arrays of bytes representing Unicode characters')

'probably hence allowing   modify  copy without damaging  original,  reason  also works  strings    Python, Strings  arrays  bytes representing Unicode characters'

# Handling emojis

In [34]:
#case 1 removing the emoji's
import re
def remove_emoji(string):
    emoji_pattern = re.compile("["
                           u"\U0001F600-\U0001F64F" # emoticons
                           u"\U0001F300-\U0001F5FF" # symbols & pictographs
                           u"\U0001F680-\U0001F6FF" # transport & map symbols
                           u"\U0001F1E0-\U0001F1FF" # flags (iOS)
                           u"\U00002702-\U000027B0"
                           u"\U000024C2-\U0001F251"
                           "]+", flags=re.UNICODE)
    return emoji_pattern.sub(r'', string)

In [35]:
remove_emoji("Hilarious 😂! The feeling of making a sale 😎, The feeling of actually fulfilling orders 😒")

'Hilarious ! The feeling of making a sale , The feeling of actually fulfilling orders '

In [36]:
# case 2 Replacing the emoji
import emoji 
print(emoji.demojize('The feeling of making a sale 😎'))

The feeling of making a sale :smiling_face_with_sunglasses:


In [37]:
import emoji 
print(emoji.demojize('Meaning of this: 😙'))

Meaning of this: :kissing_face_with_smiling_eyes:
